In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #if filename == 'kindle_reviews.csv':
            #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Importing csv file
prepro_data = pd.read_csv('./kindle_reviews.csv', sep=',')
prepro_data.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200


In [3]:
# We drop non relevant columns.
data = prepro_data.drop(columns=['Unnamed: 0', 'reviewerID', 'reviewerName'])
data['reviewText'] = data['reviewText'] + ' ' + data['summary']
data.head()

,asin,helpful,overall,reviewText,reviewTime,summary,unixReviewTime
0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",Nice vintage story,1399248000
1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",Different...,1388966400
2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",Oldie,1396569600
3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",I really liked it.,1392768000
4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",Period Mystery,1395187200


In [4]:
# Divide helpful array in two columns and trasnform all 0s in helpfultotal for 1s. 
# There wont be any problems because helpfulUp will always be 0.
helpfulValues = data['helpful'].transform(lambda n: n[1:-1].replace(" ", "").split(','))
data['helpfulUp'] = helpfulValues.transform(lambda n: int(n[0]))
data['helpfulTotal'] = helpfulValues.transform(lambda n: (int(n[1]) if int(n[1]) > 0 else 1))
data = data.drop(columns = 'helpful')
data.head()

,asin,overall,reviewText,reviewTime,summary,unixReviewTime,helpfulUp,helpfulTotal
0,B000F83SZQ,5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",Nice vintage story,1399248000,0,1
1,B000F83SZQ,4,This book is a reissue of an old one; the auth...,"01 6, 2014",Different...,1388966400,2,2
2,B000F83SZQ,4,This was a fairly interesting read. It had ol...,"04 4, 2014",Oldie,1396569600,2,2
3,B000F83SZQ,5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",I really liked it.,1392768000,1,1
4,B000F83SZQ,4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",Period Mystery,1395187200,0,1


In [5]:
# Get the maximum helpfulTotal by product and merge it with the original data.
maxReviewsProduct = data.groupby(["asin"])['helpfulTotal'].max()
maxReviewsProduct.head()
maxReviewsProduct.name = 'maxReviewsProduct'
data = data.merge(maxReviewsProduct, on='asin')
data = data.drop(columns='asin')
data.head()

,overall,reviewText,reviewTime,summary,unixReviewTime,helpfulUp,helpfulTotal,maxReviewsProduct
0,5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",Nice vintage story,1399248000,0,1,2
1,4,This book is a reissue of an old one; the auth...,"01 6, 2014",Different...,1388966400,2,2,2
2,4,This was a fairly interesting read. It had ol...,"04 4, 2014",Oldie,1396569600,2,2,2
3,5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",I really liked it.,1392768000,1,1,2
4,4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",Period Mystery,1395187200,0,1,2


In [6]:
# Create column popularity of review, which is the number of totalvotes divided by the 
# maximum totalvotes a review got on that product.
data['popularity'] = data['helpfulUp']/data['maxReviewsProduct']

# Create column acceptance of review, which is the number of upvotes divided by
# the number of totalvotes.
data['acceptance'] = data['helpfulUp']/data['helpfulTotal']

# We drop non relevant columns and see the result
# data = data.drop(columns = ['helpfulUp', 'helpfulTotal', 'maxReviewsProduct'])

data.head()

,overall,reviewText,reviewTime,summary,unixReviewTime,helpfulUp,helpfulTotal,maxReviewsProduct,popularity,acceptance
0,5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",Nice vintage story,1399248000,0,1,2,0.0,0.0
1,4,This book is a reissue of an old one; the auth...,"01 6, 2014",Different...,1388966400,2,2,2,1.0,1.0
2,4,This was a fairly interesting read. It had ol...,"04 4, 2014",Oldie,1396569600,2,2,2,1.0,1.0
3,5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",I really liked it.,1392768000,1,1,2,0.5,1.0
4,4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",Period Mystery,1395187200,0,1,2,0.0,0.0


In [7]:
# Next, we finally get our stimator of good reviews in acceptance times popularity. It is
# done with times rather than plus because we want to value more when the two terms are
# similar than when one of them is very big.
data['stimator'] = data['popularity'] * data['acceptance']

# We drop non relevant columns and see the result
#data = data.drop(columns = ['popularity', 'acceptance'])

data.head()

,overall,reviewText,reviewTime,summary,unixReviewTime,helpfulUp,helpfulTotal,maxReviewsProduct,popularity,acceptance,stimator
0,5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",Nice vintage story,1399248000,0,1,2,0.0,0.0,0.0
1,4,This book is a reissue of an old one; the auth...,"01 6, 2014",Different...,1388966400,2,2,2,1.0,1.0,1.0
2,4,This was a fairly interesting read. It had ol...,"04 4, 2014",Oldie,1396569600,2,2,2,1.0,1.0,1.0
3,5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",I really liked it.,1392768000,1,1,2,0.5,1.0,0.5
4,4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",Period Mystery,1395187200,0,1,2,0.0,0.0,0.0


In [8]:
# Importing ML libraries.

import sys
from pandas_profiling import ProfileReport

import re, string, unicodedata
import contractions
import inflect
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import svm
from sklearn.svm import SVC, SVR
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, plot_precision_recall_curve, plot_confusion_matrix,  mean_squared_error, r2_score
from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juanc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juanc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\juanc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## String prep

In [9]:
# Copy df and lower all strings.
data = data.iloc[0:2000]

#data_string['reviewText'] = data_string['reviewText'].transform(lambda n: str(n).lower())
#data_string['summary'] = data_string['summary'].transform(lambda n: str(n).lower())
#data_string[['reviewText','summary']] = data_string[['reviewText','summary']].apply(lambda n: str(n).lower())
data.head()

,overall,reviewText,reviewTime,summary,unixReviewTime,helpfulUp,helpfulTotal,maxReviewsProduct,popularity,acceptance,stimator
0,5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",Nice vintage story,1399248000,0,1,2,0.0,0.0,0.0
1,4,This book is a reissue of an old one; the auth...,"01 6, 2014",Different...,1388966400,2,2,2,1.0,1.0,1.0
2,4,This was a fairly interesting read. It had ol...,"04 4, 2014",Oldie,1396569600,2,2,2,1.0,1.0,1.0
3,5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",I really liked it.,1392768000,1,1,2,0.5,1.0,0.5
4,4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",Period Mystery,1395187200,0,1,2,0.0,0.0,0.0


In [10]:
# Methods for transforming words.

def preprocessing(words):
    """Remove non-ASCII characters from list of tokenized words"""
    p = inflect.engine()
    lm = WordNetLemmatizer()
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_word = new_word.lower()
        new_word = re.sub(r'[^\w\s]', '', new_word)
        if new_word == '':
            continue
        if word.isdigit():
            new_word = p.number_to_words(new_word)
        new_word = lm.lemmatize(new_word)
        if new_word in stopwords.words('english'):
            continue
        new_words.append(new_word)
    return new_words

In [11]:
# Convert in string and correct contractions
data['reviewText'] = data['reviewText'].astype(str).apply(contractions.fix)
data['summary'] = data['summary'].astype(str).apply(contractions.fix)

data.head()

,overall,reviewText,reviewTime,summary,unixReviewTime,helpfulUp,helpfulTotal,maxReviewsProduct,popularity,acceptance,stimator
0,5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",Nice vintage story,1399248000,0,1,2,0.0,0.0,0.0
1,4,This book is a reissue of an old one; the auth...,"01 6, 2014",Different...,1388966400,2,2,2,1.0,1.0,1.0
2,4,This was a fairly interesting read. It had ol...,"04 4, 2014",Oldie,1396569600,2,2,2,1.0,1.0,1.0
3,5,I would never read any of the Amy Brewster mys...,"02 19, 2014",I really liked it.,1392768000,1,1,2,0.5,1.0,0.5
4,4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",Period Mystery,1395187200,0,1,2,0.0,0.0,0.0


In [12]:
# Tokenize and apply preprocessing
data['reviewText'] = data['reviewText'].apply(word_tokenize).apply(preprocessing)
data['summary'] = data['summary'].apply(word_tokenize).apply(preprocessing)

data.head()

,overall,reviewText,reviewTime,summary,unixReviewTime,helpfulUp,helpfulTotal,maxReviewsProduct,popularity,acceptance,stimator
0,5,"[enjoy, vintage, book, movie, enjoyed, reading...","05 5, 2014","[nice, vintage, story]",1399248000,0,1,2,0.0,0.0,0.0
1,4,"[book, reissue, old, one, author, wa, born, 19...","01 6, 2014",[different],1388966400,2,2,2,1.0,1.0,1.0
2,4,"[wa, fairly, interesting, read, old, style, te...","04 4, 2014",[oldie],1396569600,2,2,2,1.0,1.0,1.0
3,5,"[would, never, read, amy, brewster, mystery, o...","02 19, 2014","[really, liked]",1392768000,1,1,2,0.5,1.0,0.5
4,4,"[like, period, piece, clothing, lingo, enjoy, ...","03 19, 2014","[period, mystery]",1395187200,0,1,2,0.0,0.0,0.0


In [13]:
# Duplicate data and rejoin strings.
data_intocable = data.copy()
data_intocable['reviewText'] = data_intocable['reviewText'].apply(lambda x: ' '.join(map(str, x)))
data_intocable['summary'] = data_intocable['summary'].apply(lambda x: ' '.join(map(str, x)))
data_intocable.head()

,overall,reviewText,reviewTime,summary,unixReviewTime,helpfulUp,helpfulTotal,maxReviewsProduct,popularity,acceptance,stimator
0,5,enjoy vintage book movie enjoyed reading book ...,"05 5, 2014",nice vintage story,1399248000,0,1,2,0.0,0.0,0.0
1,4,book reissue old one author wa born 1910 era s...,"01 6, 2014",different,1388966400,2,2,2,1.0,1.0,1.0
2,4,wa fairly interesting read old style terminolo...,"04 4, 2014",oldie,1396569600,2,2,2,1.0,1.0,1.0
3,5,would never read amy brewster mystery one real...,"02 19, 2014",really liked,1392768000,1,1,2,0.5,1.0,0.5
4,4,like period piece clothing lingo enjoy mystery...,"03 19, 2014",period mystery,1395187200,0,1,2,0.0,0.0,0.0


In [14]:
# Separate variables.
x_data, y_data = data_intocable['reviewText'], data_intocable['stimator']

# Transform to count
#count = CountVectorizer()
#x_dummy = count.fit_transform(x_data)
#print(x_dummy)

# Transform to binary.
#dummy = CountVectorizer(binary=True)
#x_dummy = dummy.fit_transform(x_data)
#print(x_dummy)

# Transform to tfidf.
tfidf_vectorizer=TfidfVectorizer() 
x_dummy=tfidf_vectorizer.fit_transform(x_data)
print(x_dummy)

  (0, 10785)	0.05372466707943299
  (0, 7565)	0.11848062887441038
  (0, 9025)	0.05127417949108244
  (0, 4746)	0.06878885139063383
  (0, 11521)	0.24938018773760376
  (0, 12311)	0.236594185919034
  (0, 12673)	0.07439473154410793
  (0, 6277)	0.21473637143894875
  (0, 6031)	0.236594185919034
  (0, 5290)	0.22752237325751853
  (0, 8484)	0.19562238769097395
  (0, 7669)	0.24938018773760376
  (0, 12573)	0.12244410123125878
  (0, 1339)	0.159123085617454
  (0, 9807)	0.11100018112693404
  (0, 6439)	0.1767700997233728
  (0, 9931)	0.236594185919034
  (0, 10465)	0.276358737558453
  (0, 6219)	0.1880175446715145
  (0, 11360)	0.10416001170914183
  (0, 12005)	0.181894997739397
  (0, 12243)	0.1038586072997268
  (0, 8437)	0.10783480778105436
  (0, 9037)	0.08358948455843732
  (0, 3669)	0.09064666587071908
  :	:
  (1998, 6543)	0.020596871525867283
  (1998, 11469)	0.02184497767241121
  (1998, 7542)	0.02782650968635662
  (1998, 7833)	0.08866397774831258
  (1998, 9025)	0.04195328005519683
  (1998, 4746)	0.018761

Después de probar con cada uno de los tipos de transformaciones, pudimos concluir que tfidf es la mejor manera de representar los datos. Esto se concluyó a partir de los valores que daban los coeficientes de los algoritmos de ML.

In [15]:
# Transform the array to a dataframe to help with analysis.
x_dummy_np = x_dummy.todense()
x_dummy_df = pd.DataFrame(x_dummy_np)
print(x_dummy_df.shape)
print(type(x_dummy_df))

(2000, 12809)
<class 'pandas.core.frame.DataFrame'>


In [16]:
# Splitting to make tests
x_train, x_test, y_train, y_test = train_test_split(x_dummy_df, y_data, test_size=0.2, random_state=10)

## Linear regression

In [17]:
# Linear regression and predictions
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# The coefficients
print('Coefficients: \n', model.coef_)
# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

Coefficients: 
 [-0.17107896 -0.0309171   0.02663457 ... -0.00292979 -0.07569468
  0.        ]
Mean squared error: 0.09
Coefficient of determination: -0.31


## SVR

In [18]:
# SVR
model = svm.SVR()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# The coefficients
print('Coefficients: \n', model.dual_coef_)
# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

Coefficients: 
 [[-0.18543221 -0.12152374  0.14872404 ... -0.20353275 -0.13045707
  -0.06054116]]
Mean squared error: 0.07
Coefficient of determination: 0.04


## Ridge Regression

In [20]:
# Ridge regression and predictions
model = Ridge()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# The coefficients
print('Coefficients: \n', model.coef_)
# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

Coefficients: 
 [ 0.         -0.01444583 -0.00823767 ...  0.02641407 -0.02308521
  0.        ]
Mean squared error: 0.06
Coefficient of determination: 0.08


## Lasso regression

In [21]:
# Lasso regression and predictions
model = Lasso(alpha=0.5)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# The coefficients
print('Coefficients: \n', model.coef_)
# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))


Coefficients: 
 [ 0. -0. -0. ...  0. -0.  0.]
Mean squared error: 0.07
Coefficient of determination: -0.00
